(sec:newton)=
# Verfahren zweiter Ordnung

## Motivation 

## Newton Verfahren

### Konvergenzgeschwindigkeit

### Regularisierung des Verfahrens


## Quasi-Newton Verfahren

### DFP 

### BFGS

### Limited-memory BFGS

## Zusammenfassung der Verfahren erster und zweiter Ordnung